In [5]:
import pygame
from ipynb.fs.full.bus import Bus
import math
import pickle
import copy

class GameInformation:
    def __init__(self):
        self.rights = 0
        self.lefts = 0
        self.ups = 0
        self.downs = 0
        self.total_distance = 0
        self.pickups = 0  # Initialize pickups count
        self.finish = False
        self.unique_positions = set()

class Game:
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)
    BLUE = (0, 0, 255)
    GREEN = (0, 255, 0)

    def __init__(self, window, window_width, window_height, pickup_locations):
        self.window = window
        self.window_width = window_width
        self.window_height = window_height
        self.cell_size = 10
        self.grid_width = window_width // self.cell_size
        self.grid_height = window_height // self.cell_size
        self.map_grid = self.generate_map_layout()
        self.school_location = (self.grid_width // 2, self.grid_height // 2)
        self.original_pickup_locations = pickup_locations
        self.pickup_locations = self.original_pickup_locations.copy()
        self.bus = Bus(self.school_location[0] * self.cell_size, self.school_location[1] * self.cell_size)
        self.game_info = GameInformation()
    
    def generate_map_layout(self):
        map_grid = []
        for y in range(self.grid_height):
            row = []
            for x in range(self.grid_width):
                # Mark cells as roads more liberally to make them logically thicker
                if x % 10 < 2 or y % 10 < 2:  # Adjusted to make roads twice as thick
                    row.append(True)
                elif ((x % 5 < 2 and y < self.grid_height) or 
                      (y % 5 < 2 and x < self.grid_width)):
                    row.append(True)  # Adjust as needed for secondary roads
                else:
                    row.append(False)
            map_grid.append(row)
        return map_grid

        
    def is_valid_move(self, x, y):
        cell_x, cell_y = x // self.cell_size, y // self.cell_size
        return 0 <= cell_x < self.grid_width and 0 <= cell_y < self.grid_height and self.map_grid[cell_y][cell_x]

    def move_bus(self, direction):
        dx, dy = 0, 0
        if direction == "left":
            dx = -1
        elif direction == "right":
            dx = 1
        elif direction == "up":
            dy = -1
        elif direction == "down":
            dy = 1

        new_x, new_y = self.bus.x + (dx * self.cell_size), self.bus.y + (dy * self.cell_size)
        if self.is_valid_move(new_x, new_y):
            self.bus.x, self.bus.y = new_x, new_y
            self.game_info.total_distance += 1
            self.game_info.unique_positions.add((self.bus.x, self.bus.y))

    def check_student_pickup(self):
        bus_grid_pos = (self.bus.x // self.cell_size, self.bus.y // self.cell_size)
        for loc in self.pickup_locations[:]:  # Safely iterate through a copy
            if bus_grid_pos == loc:
                print(f"Student picked up at {loc}.")
                self.pickup_locations.remove(loc)  # Remove the collected pickup location
                self.game_info.pickups += 1

    def check_finish(self):
        if self.game_info.pickups == len(self.pickup_locations) and (self.bus.x // self.cell_size, self.bus.y // self.cell_size) == self.school_location:
            self.game_info.finish = True
        return self.game_info.finish

    def update(self, direction):
        self.move_bus(direction)
        self.check_student_pickup()
        finish = self.check_finish()
        return finish

    def reset_for_new_genome(self):
        """Resets the game state for a new genome evaluation."""
        self.pickup_locations = copy.deepcopy(self.original_pickup_locations)  # Reset pickup locations
        self.game_info = GameInformation()  # Reset game information
        self.bus.x, self.bus.y = self.school_location[0] * self.cell_size, self.school_location[1] * self.cell_size  # Reset bus position
        # Reset other necessary states here, if any


    def render(self):
        # Clear the screen
        self.window.fill(self.BLACK)
        # Draw the map grid
        for y in range(self.grid_height):
            for x in range(self.grid_width):
                if self.map_grid[y][x]:  # If the cell is marked as road
                    # Calculate the rectangle for the road
                    rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size * 2, self.cell_size * 2)
                    pygame.draw.rect(self.window, self.WHITE, rect)
                else:
                    # Calculate the rectangle for non-road
                    rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
                    pygame.draw.rect(self.window, self.BLACK, rect)
                    
        # Draw the school location in red
        school_rect = pygame.Rect(self.school_location[0] * self.cell_size, self.school_location[1] * self.cell_size, self.cell_size*2, self.cell_size*2)
        pygame.draw.rect(self.window, self.RED, school_rect)
        
        # Draw each pickup location in blue
        for loc in self.pickup_locations:
            pickup_rect = pygame.Rect(loc[0] * self.cell_size, loc[1] * self.cell_size, self.cell_size*2, self.cell_size*2)
            pygame.draw.rect(self.window, self.BLUE, pickup_rect)
        
        # Draw the bus in green
        bus_rect = pygame.Rect(self.bus.x, self.bus.y, self.cell_size, self.cell_size)
        pygame.draw.rect(self.window, self.GREEN, bus_rect)
        pygame.display.flip()


    @staticmethod
    def distance_to_closest_pickup(bus_position, pickup_locations):
        import math
        if not pickup_locations:
            return float('inf')
        return min(math.hypot(bus_position[0] - p[0], bus_position[1] - p[1]) for p in pickup_locations)

    @staticmethod
    def distance_to_school(bus_position, school_position):
        import math
        return math.hypot(bus_position[0] - school_position[0], bus_position[1] - school_position[1])

    
    def calculate_fitness(self):
        fitness = 0
        exploration_bonus = 1
        pickup_bonus = 200
        movement_tax = 0.01
        stagnation_penalty = 100
        bus_position = (self.bus.x, self.bus.y)
        
        # Encourage exploration
        fitness += len(self.game_info.unique_positions) * exploration_bonus
    
        # Heavy reward for each pickup
        fitness += self.game_info.pickups * pickup_bonus
    
        # Penalize for stagnation
        if self.game_info.total_distance < 1000:
            fitness -= stagnation_penalty
    
        # If all pickups are collected, give a large bonus and subtract the distance to the school
        if self.game_info.pickups == len(self.pickup_locations):
            fitness += large_bonus
            fitness -= self.distance_to_school(bus_position, self.school_location)
        else:
            # Penalize based on distance to the closest uncollected pickup
            fitness -= self.distance_to_closest_pickup(bus_position, self.pickup_locations)

        print(fitness)
        print(self.distance_to_school(bus_position, self.school_location))
        print(bus_position)
    
        return fitness


    def run(self, direction):
        finish = self.update(direction)
        self.render()
        return finish


